In [7]:
import pandas as pd
import numpy as np
import glob
from svv_read import Sub_SVV
import matplotlib.pyplot as plt

In [2]:
%matplotlib auto
pd.options.display.max_rows = None

Using matplotlib backend: MacOSX


In [72]:
# create data frame for results3
df = pd.DataFrame()

readfolder = '../DATA/EXP2/'
list_subject = glob.glob(readfolder+'/S*')
# list_subject

## check following outliers

 - check S50: -20º values (bucket moved during test)
 - check S45: very high angle and very short RT
 - check S46: similar S45
 - check S47: low angles

In [73]:
for sub in list_subject:
    SUB = Sub_SVV(sub)
    df = df.append(SUB.create_table(), ignore_index=True)


In [74]:
# snb.stripplot(x='freq', y='SVV', data = df)
# snb.stripplot(x='freq', y='SVV', data = df[(df['stim']=='SH') & (df['SVV']>2.5)])

## check for Subjective Visual Vesrtical over 2.5º in control condition

In [75]:
outliers = [(np.abs(df.loc[i,'SVV'])>2.5) and (df.loc[i,'stim']=='SH') for i in df.index]

In [77]:
bad_id = df[outliers].id.unique()

In [78]:
df.insert(len(df.columns), 'Outliers', df['id'].isin(bad_id))

## graphical representation of SVV data

In [94]:
plt.figure()
idx = (df['stim']=='SH') | (df['stim']=='DC')
plt.boxplot(df[idx].SVV)

{'whiskers': [<matplotlib.lines.Line2D at 0x11d957a50>,
 'caps': [<matplotlib.lines.Line2D at 0x11d9614d0>,
 'boxes': [<matplotlib.lines.Line2D at 0x11d8fee50>],
 'medians': [<matplotlib.lines.Line2D at 0x11d961f10>],
 'fliers': [<matplotlib.lines.Line2D at 0x11d5c72d0>],
 'means': []}

In [82]:
plt.subplot(2,1,1)
plt.plot(df.freq, df.SVV, 'k.')
plt.plot(df[df.Outliers].freq, df[df.Outliers].SVV, 'r.', ms=15)

array(['S45', 'S47', 'S46'], dtype=object)

In [81]:
df[df['id']=='S45']

,id,trial,stim,freq,rep,Angle_init,SVV,Reaction_time,Outliers
704,S45,T08,MF,20.0,T0,-20.726651,3.540015,4671.0,True
705,S45,T10,SH,0.0,T0,-24.807628,3.892372,5020.0,True
706,S45,T16,AC,160.0,T0,-19.441652,4.591681,4847.0,True
707,S45,T14,MF,20.0,T0,-23.833130,4.150204,4419.0,True
708,S45,T20,MX,60.0,T0,-21.075283,3.658050,2716.0,True
709,S45,T13,AC,60.0,T1,-21.310566,4.822767,3626.0,True
710,S45,T09,DC,0.0,T1,-20.872379,3.860954,3678.0,True
711,S45,T06,MF,160.0,T0,-24.162784,4.770550,5257.0,True
712,S45,T22,MF,60.0,T0,-22.096951,3.569716,2516.0,True
713,S45,T17,AC,120.0,T0,-24.550745,5.065922,4710.0,True


In [ ]:
outliers